# Anotaciones Label Studio

In [1]:
from src.data.coco_class import *

In [2]:
from helper.almaMedical_helper import getDicomPathFiles, getAnnotationDicoms, getAnnotations

In [3]:
path_dataset = './notebooks/dataset/FILESET/'

In [9]:
def create_COCO(dicom_files):
    coco_ds = COCO(set_HIBA=True)
    coco_ds.setInfo(Info(version="1.0.0", description="Spinogram_Dataset"))
    coco_ds.addCategories(Categories(category_id=1, name='spine', supercategory='spine',
                                     keypoints=['C2OT', 'C1AE', 'C1PE', 'C2CE', 'C2AI', 'C2PI', 'C7AS', 'C7PS', 'C7CE',
                                                'C7AI', 'C7PI', 'T1AS', 'T1PS', 'T1CE', 'T1AI', 'T1PI', 'T5AS', 'T5PS',
                                                'T12A', 'T12P', 'L1AS', 'L1PS', 'L4AS', 'L4PS', 'L4AI', 'L4PI', 'S1AS',
                                                'S1MI', 'S1PS', 'F1HC', 'F2HC'],
                                     skeleton=[]))

    annotationNumber = 1

    annotation_dicoms = getAnnotationDicoms(dicom_files)

    for annotation_dicom in annotation_dicoms:

        fileId, fileName, width, height, keypoints = getAnnotations(annotation_dicom, dicom_files)

        cocoImage = ImageCOCO(image_id=fileId, width=width, height=height,
                              file_name=fileName, license_id=coco_ds.getLicense().getId())
        coco_ds.addImage(cocoImage)

        # Create annotation for image
        annotation = Annotation(annotation_id=annotationNumber, image_id=cocoImage.getId(),
                                category_id=1, keypoints=[coordinate for row in keypoints for coordinate in row],
                                num_keypoints=keypoints.shape[0])

        coco_ds.addAnnotation(annotation)

        annotationNumber += 1

    return coco_ds

In [10]:
getDicomPathFiles(path_dataset)

['./notebooks/dataset/FILESET/1036487/0/0/0',
 './notebooks/dataset/FILESET/1036487/0/0/1',
 './notebooks/dataset/FILESET/1036487/0/1/4',
 './notebooks/dataset/FILESET/1036487/0/2/2',
 './notebooks/dataset/FILESET/1036487/0/3/3']

In [11]:
annotation_dicoms = getAnnotationDicoms(getDicomPathFiles(path_dataset))

In [12]:
annotation_dicoms

['./notebooks/dataset/FILESET/1036487/0/1/4']

In [13]:
dataset_test = create_COCO(getDicomPathFiles(path_dataset))

In [14]:
dataset_test.exportJson('./notebooks/dataset/', 'test.json')